# Первый блокнот

In [63]:
print(1)

1


In [64]:
import pandas as pd
import seaborn as sns
import numpy as np
from math import isclose
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import pymorphy2
import re

In [65]:
df = pd.read_csv('data.csv')
df.head()

,id,choosing_readiness_new_meetings,choosing_meeting_format,choosing_personality_type,choosing_hobbies,tell_what_do_you_do,tell_expectations,choosing_expectations,choosing_zodiac_signs,choosing_gender,tell_hobbies
0,2,0,0,0,", ведение соцсетей и блогов, путешествия",покшула п лулатутатат алалатетатпь,владалалалвтту душуовтататат шаовоаокоаоа шаоа...,1.00,0,0,алклал лалулуиатату далутутададв
1,4,0,0,3,", техника и автомобили",цццццццццццццццццццццц,цццццццццццццццццццццццц,0.50,4,1,уццццццццццццццццццццццц
2,5,0,0,0,", ведение соцсетей и блогов",мрукмтукру к8урк укмт ук8у мтцу мрукзщмо ум,мулкомр укгмр укр зкумр гуркм у км,1.00,0,0,алтуйшкмумтумк кшмтук м1куругмр укмрукмо умтцут
3,6,1,0,3,", музыка, видеоигры",продукт аналитик,делал интересные и долгосрочные проекты,0.75,4,0,"люблю качаться в зале, изучать психологию"
4,8,1,0,3,", музыка, игра на музыкальных инструментах",NaN,NaN,0.75,0,0,NaN


In [66]:
df.shape

(185, 11)

In [67]:
df.columns

Index(['id', 'choosing_readiness_new_meetings', 'choosing_meeting_format',
       'choosing_personality_type', 'choosing_hobbies', 'tell_what_do_you_do',
       'tell_expectations', 'choosing_expectations', 'choosing_zodiac_signs',
       'choosing_gender', 'tell_hobbies'],
      dtype='object')

In [68]:
text_choosing_columns = ['choosing_readiness_new_meetings', 'choosing_meeting_format', 'choosing_personality_type', 'choosing_expectations', 'choosing_zodiac_signs', 'choosing_gender']
text_multi_choosing_columns = ['choosing_hobbies']
text_tell_columns = ['tell_what_do_you_do', 'tell_expectations', 'tell_hobbies']

### text_choosing_columns
```python
available_readiness_new_meetings = ["Да!", "Зависит от настроения", "Не люблю знакомиться с новыми людьми"]
available_expectations = ["100% фана", "75% фана, немного серьезности", "50% фана, 50% серьезности",
                          "25% фана, 75% серьезности", "0% фана, только деловые беседы"]
available_meeting_format = ["Готов вживую", "Только онлайн"]
available_zodiac_signs = ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец",
                          "Козерог", "Водолей", "Рыбы"]
available_personality_type = ["Стратег", "Учёный", "Командир", "Полемист",
                              "Активист", "Посредник", "Тренер", "Борец",
                              "Администратор", "Защитник", "Менеджер", "Консул",
                              "Виртуоз", "Артист", "Делец", "Развлекатель"]
available_gender = ["Мужской", "Женский", "Пропустить"]
```
### text_multi_choosing_columns
```python
available_hobbies = ["Ведение соцсетей и блогов", "Путешествия", "Музыка", "Книги", "Кино и сериалы", "Видеоигры",
                     "Игра на музыкальных инструментах", "Кулинария", "Искусство и рукоделие", "Коллекционирование",
                     "Техника и автомобили"]
```

In [69]:
decisive_columns = ['choosing_readiness_new_meeting','choosing_meeting_format', 'choosing_gender']
categorical_columns = ['choosing_personality_type',  'choosing_zodiac_signs']
num_columns = ['choosing_expectations']

In [71]:
df.isna().sum()

id                                  0
choosing_readiness_new_meetings     0
choosing_meeting_format             0
choosing_personality_type           0
choosing_hobbies                    0
tell_what_do_you_do                50
tell_expectations                  50
choosing_expectations               0
choosing_zodiac_signs               0
choosing_gender                     0
tell_hobbies                       50
dtype: int64

In [72]:
morph = pymorphy2.MorphAnalyzer()

def is_strange(text):
    if pd.isna(text) or not text.strip():
        return True
    if bool(re.search(r'(.)\1{3,}', text)):
        return True
    
    words = text.lower().split()
    analyzed_words = [morph.parse(word)[0] for word in words]

    non_existent_words = sum(1 for word in analyzed_words if 'UNKN' in word.tag)
    words_with_digits = sum(1 for word in words if any(char.isdigit() for char in word))
    return non_existent_words >= len(words) / 2 or words_with_digits > 0

for col in text_tell_columns:
    df[col] = df[col].apply(lambda x: pd.NA if is_strange(x) else x)

is_na_in_tell_columns = df[text_tell_columns].isna().any(axis=1)
if is_na_in_tell_columns.any():
    df.loc[is_na_in_tell_columns, text_tell_columns] = pd.NA
df.shape

(185, 11)

In [73]:
df.head(10)

,id,choosing_readiness_new_meetings,choosing_meeting_format,choosing_personality_type,choosing_hobbies,tell_what_do_you_do,tell_expectations,choosing_expectations,choosing_zodiac_signs,choosing_gender,tell_hobbies
0,2,0,0,0,", ведение соцсетей и блогов, путешествия",покшула п лулатутатат алалатетатпь,владалалалвтту душуовтататат шаовоаокоаоа шаоа...,1.00,0,0,алклал лалулуиатату далутутададв
1,4,0,0,3,", техника и автомобили",<NA>,<NA>,0.50,4,1,<NA>
2,5,0,0,0,", ведение соцсетей и блогов",<NA>,<NA>,1.00,0,0,<NA>
3,6,1,0,3,", музыка, видеоигры",продукт аналитик,делал интересные и долгосрочные проекты,0.75,4,0,"люблю качаться в зале, изучать психологию"
4,8,1,0,3,", музыка, игра на музыкальных инструментах",<NA>,<NA>,0.75,0,0,<NA>
5,7,1,0,0,", видеоигры, кино и сериалы, книги",data scientist в сбере,не знаю не знаю не знаю,0.50,4,0,из видеоигр люблю сюжетные рпг и различные сим...
6,18,1,0,2,", музыка, игра на музыкальных инструментах, ви...",студент питерской вышки,"не состоял, ничего не ожидаю :р",0.75,0,0,я вообще ничего не делаю. занимаю ничегонедела...
7,9,0,0,1,", путешествия, книги, игра на музыкальных инст...",data science в банке,"с людьми по интересам общаться, что-то новое у...",0.50,3,1,люблю ходить по музеям и изучать искусство. лю...
8,12,2,0,1,", игра на музыкальных инструментах, техника и ...",<NA>,<NA>,0.50,7,0,<NA>
9,11,1,0,8,", музыка, игра на музыкальных инструментах, кн...","ml-стажер в небольшой компании, занимаюсь эксп...",побаловаться поприкалываться пообщаться с людь...,0.50,10,1,описываю намного подробней


In [74]:
df.isna().sum()

id                                  0
choosing_readiness_new_meetings     0
choosing_meeting_format             0
choosing_personality_type           0
choosing_hobbies                    0
tell_what_do_you_do                89
tell_expectations                  89
choosing_expectations               0
choosing_zodiac_signs               0
choosing_gender                     0
tell_hobbies                       89
dtype: int64

In [75]:
df_encoded = pd.get_dummies(df[categorical_columns], columns=categorical_columns)

In [76]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize_text(text):
    return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])

for col in text_tell_columns:
    df[col] = df[col].fillna('').apply(lemmatize_text)

In [77]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrices = [tfidf_vectorizer.fit_transform(df[col]) for col in text_tell_columns]

tfidf_hobbies = TfidfVectorizer()
tfidf_hobbies_matrix = tfidf_hobbies.fit_transform(df['choosing_hobbies'])

In [82]:
combined_tfidf_matrix = hstack(tfidf_matrices + [tfidf_hobbies_matrix])
final_matrix = hstack([df_encoded, combined_tfidf_matrix])

final_df = pd.DataFrame.sparse.from_spmatrix(final_matrix)

In [81]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(final_df)

df['cluster'] = clusters
df.head(20)

,id,choosing_readiness_new_meetings,choosing_meeting_format,choosing_personality_type,choosing_hobbies,tell_what_do_you_do,tell_expectations,choosing_expectations,choosing_zodiac_signs,choosing_gender,tell_hobbies,cluster
0,2,0,0,0,", ведение соцсетей и блогов, путешествия",покшуть п лулатутатат алалатетатпать,владалалалвтта душуовтататат шаовоаокоаоа шаоа...,1.00,0,0,алкласть лалулуиатат далутутададть,3
1,4,0,0,3,", техника и автомобили",,,0.50,4,1,,2
2,5,0,0,0,", ведение соцсетей и блогов",,,1.00,0,0,,3
3,6,1,0,3,", музыка, видеоигры",продукт аналитик,делать интересный и долгосрочный проект,0.75,4,0,"любить качаться в зале, изучать психология",2
4,8,1,0,3,", музыка, игра на музыкальных инструментах",,,0.75,0,0,,2
5,7,1,0,0,", видеоигры, кино и сериалы, книги",data scientist в сбер,не знать не знать не знать,0.50,4,0,из видеоигра любить сюжетный рпг и различный с...,3
6,18,1,0,2,", музыка, игра на музыкальных инструментах, ви...",студент питерский вышка,"не состоял, ничего не ожидать :р",0.75,0,0,я вообще ничего не делаю. занимать ничегонедел...,2
7,9,0,0,1,", путешествия, книги, игра на музыкальных инст...",data science в банк,"с человек по интерес общаться, что-то новый уз...",0.50,3,1,любить ходить по музей и изучать искусство. лю...,2
8,12,2,0,1,", игра на музыкальных инструментах, техника и ...",,,0.50,7,0,,2
9,11,1,0,8,", музыка, игра на музыкальных инструментах, кн...","ml-стажёр в небольшой компании, заниматься экс...",побаловаться поприкалываться пообщаться с чело...,0.50,10,1,описывать намного подробный,4
